In [2]:
!nvidia-smi

Thu Dec  3 05:03:21 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
os.chdir('/content/drive/My Drive/GitHub/mitochondria_segmentation/src')
print(os.getcwd())
#print(os.listdir())

/content/drive/My Drive/GitHub/mitochondria_segmentation/src


In [ ]:
!python train.py

Data loaded.
device: cuda:0
train.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float32)
train.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  lables = torch.tensor(lables, dtype=torch.long)
[271, loss: 0.010238]
7.0 minutes per epoch.
[272, loss: 0.010217]
4.0 minutes per epoch.
[273, loss: 0.010196]
4.0 minutes per epoch.
[274, loss: 0.010176]
4.0 minutes per epoch.
[275, loss: 0.010156]
4.0 minutes per epoch.
[276, loss: 0.010136]
4.0 minutes per epoch.
[277, loss: 0.010116]
4.0 minutes per epoch.
[278, loss: 0.010096]
4.0 minutes per epoch.
[279, loss: 0.010076]
4.0 minutes per epoch.
[280, loss: 0.010056]
4.0 minutes per epoch.
[

In [ ]:
!python predict.py

In [ ]:
!python messure.py

{'TP': 583879, 'TN': 11058023, 'FP': 27791, 'FN': 126787, 'acc': 0.9868962605794269, 'sn': 0.9545653702159644, 'sp': 0.9886643581786367, 'rc': 0.8215941103134231, 'f1': 0.8831023275041561, 'jac': 0.7906743385193711, 'dice': 0.8831023280013546}


In [ ]:
!cat predict.py

# -*- coding: utf-8 -*-
"""
Created on Tue Nov 17 22:12:03 2020

@author: zll
"""


import cv2 as cv
import torch
import numpy as np
import os
from model import U_Net


def seg_img(img_file, model, device):
    '''returns prediction of numpy 2darray of uint8 in main memory.'''
    img = cv.imread(img_file, cv.IMREAD_GRAYSCALE)
    # image size is 768 * 1024, no need padding
    img = img / 255
    img = img.reshape((1, 1, img.shape[0], img.shape[1]))
    inputs = torch.from_numpy(img)
    inputs = inputs.float()
    inputs = inputs.to(device)
    outputs = model(inputs)
    outputs = outputs.to('cpu')
    outputs = outputs[0, 1, :, :]
    outputs = np.uint8(outputs >= 0.5)
    outputs *= 255
    return outputs


if __name__ == '__main__':
    model_path = './param_32/param480.pkl'
    out_path = '../pred_5/'
    img_path = '../../mito_imgs/test/images/'
    device = 'cuda:0'
    img_files = sorted(os.listdir(img_path))
    unet = U_Net()
    unet.eval()
    unet.load_state_dict(torch.l